### First problem: How to determined the value of a variable in a computer program

**Answer: LEGB [local, enclosing, global, builtin]**

*Access global variable inside a scope*

In [2]:
x = "global"

def foo():
    print("x inside:", x)


foo()
print("x outside:", x)

x inside: global
x outside: global


*Modify a global variable inside a scope*

In [3]:
x = "global"

def foo():
    global x  # to be commented
    x += '123'
    print("x inside:", x)


foo()
print("x outside:", x)

x inside: global123
x outside: global123


*Use global variable together with local variable*

In [6]:
x = "global "

def foo():
    global x
    y = "local"
    x = x * 2
    print(x)
    print(y)

foo()

global global 
local


*Use local vairable outside scope*

In [4]:
# def foo():
#     y = "local"


# foo()
# print(y)

<font color='darkred'>**Warning: Avoid Global Variables**</font>                    
Programs with global variables are difficult to maintain and extend because you can no longer view each function as a “black box” that simply receives arguments and returns a result.         

When functions modify global variables, it becomes more difficult to understand the effect of function calls. As programs get larger, this difficulty mounts quickly. Instead of using global variables, use function parameter variables and return values to transfer information from one part of a program to another. 

Global constants, however, are fine. You can place them at the top of a Python source file and access (but not modify) them in any of the functions in the file. Do not use a global declaration to access constants.       

*Nested function - nested scope*

In [7]:
def foo():
    x = 123
    def bar():
        print(x)
    return bar

In [8]:
f = foo()
f()

123


*Memorize nonlocal variable in `.__closure__` when it is* **necessary**

In [18]:
def foo():
    x = 123
    def bar():
        nonlocal x
        x += 1
        print(x)
    return bar

In [19]:
f = foo()

In [20]:
f()
f.__closure__[0].cell_contents

124


124

*Incase nonlocal variable is not nessary for closure function*

In [22]:
def foo():
    x = 123
    def bar():
        print(555)    # you don't have to memroize x inside bar function
    return bar

In [23]:
f = foo()
f()
f.__closure__ is None

555


True

### Nested function & Nested scope

In [12]:
def outer(a):
    b = a                             # b is local variable for outer, a is also a local variable for outer
    def inner():
        c = 3                         # c is local variable of inner, b is non-local variable for inner
        def inner_inner(b):           # new local variable b is defined, b is local for inner_inner, but is not the one defined in outer
            r = b+c                   # r is local variable for inner_inner, c is non-local variable for inner_inner
            return b+c
        return inner_inner
    return inner
foo = outer(10)
bar = foo()

In [13]:
foo.__closure__ is None, bar.__closure__[0].cell_contents

(True, 3)

*Use variable `a` inside `inner`*

In [14]:
def outer(a):
    b = a                            
    def inner():
        c = 3 
        def inner_inner(b):
            r = b+ c + a # use a 
            return b+c
        return inner_inner
    return inner
foo = outer(10)
bar = foo()

In [15]:
foo.__closure__ is None, bar.__closure__[0].cell_contents

(False, 10)

In [16]:
bar(4)

7

*Modify `a` in `inner_inner`*

In [17]:
def outer(a):
    b = a                            
    def inner():
        c = 3 
        def inner_inner(b):
            a += 1
            r = b+ c + a # use a 
            return b+c
        return inner_inner
    return inner
foo = outer(10)
bar = foo()

In [18]:
foo.__closure__ is None, bar.__closure__[0].cell_contents

(True, 3)

In [19]:
# bar(4)

*Although `a` is in outer scope of outer scope of `inner_inner`, it is found in nonlocal space, `__closure__` attribute*

In [20]:
def outer(a):
    b = a                            
    def inner():
        c = 3 
        def inner_inner(b):
            nonlocal a
            a += 1
            r = b + c + a # use a 
            return b+c
        return inner_inner
    return inner
foo = outer(10)
bar = foo()

In [21]:
foo.__closure__ is None, bar.__closure__[0].cell_contents

(False, 10)

In [22]:
bar(4)

7

### Modify nonlocal variable

*Functions are objects, objects have attributes and methods*

In [23]:
def abc():
    pass

setattr(abc,'haha', 'cameldata666')
setattr(abc,'haha_func', lambda x:print(x+1))

print(abc.haha)
abc.haha_func(333)

cameldata666
334


In [24]:
def sample():
    n = 0
    
    # Closure function
    def func():
        print('n=', n)
        
    # Accessor methods for n
    def get_n():
        return n
    def set_n(value):
        nonlocal n
        n = value
        
    # Attach as function attributes
    setattr(func, 'get_n', get_n)
    setattr(func,'set_n', set_n)

    return func

*`n` is defined in factory function `sample`*

In [25]:
sample_func = sample()

In [26]:
sample_func()

n= 0


*Nonlocal variblae `n` modified in `set_n` function*

In [27]:
sample_func.set_n(10)

In [28]:
sample_func()

n= 10


State: game              
Function: with_user_type            

我必须知道你用with_user_type 得state，才能决定你怎么用with_user_type函数                
如果你非要用。。。with_user_type...且被装饰得函数使用得状态还是一直变，那你不如写循环啊    
或者啊，被装饰得函数，要有修改nonlocal variable  - `game`的能力

### Return function, why:
1. Outter function is like a factory, manufacuring functions base on some conditions
2. Inner functions is a specific function to be used directly in business
**The logic of inner function frequently changes overtime, you need to modify code frequently**

**Return / manufacture functions**

In [29]:
# plura exmaple

### A decorator is just a closure

*Only admin get and put food in this store*

In [30]:
class Store(object):
    
    def get_food(self, username):
        if username != 'admin':
            raise Exception("This user is not allowed to get food")
        print('You got it')
    
    def put_food(self, username):
        if username != 'admin':
            raise Exception("This user is not allowed to put food")
        print('You modify it')

In [31]:
s = Store()
s.get_food('admin')

You got it


#### Try think about it

1. Passin a function (`f`) -> return another function (`wrapper`)
2. You get another function (`wrapper`)
3. When you call `wrapper`, `f` is also called
4. Calling `wrapper` complete all functionalities of `f`
5. Calling `wrapper` = Calling `f`
6. No, calling `wrapper` >= `f`


In [32]:
def check_is_admin(f):
    def wrapper(*args, **kwargs): # -> Everything is an [object]
        if kwargs.get('username') != 'admin':
            raise ValueError('User Premission denied!')
        return f(*args, **kwargs)
    return wrapper # -> you return this object

In [33]:
class Store(object):
    
    @check_is_admin
    def get_food(self, username):
        print('You got it')
    
    @check_is_admin
    def put_food(self, username):
        print('You modify it')

In [34]:
s = Store()
s.get_food(username = 'admin')

You got it


#### Tips 1 - equivalent

```python
@check_is_admin <=====> check_is_admin(get_food)
```

#### Tips 2 - no brakets/braces

`func(*args, **kwargs)` means you call the function
```python
class Store(object):
    
    @check_is_admin
    def get_food(self, username):
```

#### Tips 3 - Often used, but not always

#### Tips 4 - Closure
*f is local to check_is_admin*    
*f is nonlocal to wrapper*

In [35]:
def some_function():
    pass

check_is_admin(some_function).__closure__[0]

<cell at 0x0000017D02421850: function object at 0x0000017D024474C0>

In [36]:
@check_is_admin
def some_function():
    pass

some_function.__closure__[0]

<cell at 0x0000017D02421490: function object at 0x0000017D024471F0>

```python
def check_user_is_not(username):
    def user_check_decorator(f):
        def wrapper(*args, **kwargs):
            if kwargs.get('username') == username:
                raise Exception("This user is not allowed to get food")
            return f(*args, **kwargs)
        return wrapper
    return user_check_decorator

```

#### You are <font color='darkred'>**calling**</font> the outer most function `@check_user_is_not()`
```python
class Store(object):
    @check_user_is_not("admin")
    @check_user_is_not("user123")
    def get_food(self, username, food):
        return self.storage.get(food)
        
```

#### You get a decorator function `user_check_decorator`
```python
def check_user_is_not(username):
    def user_check_decorator(f):
        ............
        ............
        ............
        ............
        ............
    return user_check_decorator

```

#### Pass function f to decorator function, returns `wrapper` function

#### Calling wrapper function,the original function `f` is also called
```python
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
    xxxxxxxxxxxxxxxxxxxxxx:
        def wrapper(*args, **kwargs):
            if kwargs.get('username') == username:
                raise Exception("This user is not allowed to get food")
            return f(*args, **kwargs)
        xxxxxxxxxxxxxxxxxxxxxxxxxxx
    xxxxxxxxxxxxxxxxxxxxxxxxxxx
```

#### Template

In [106]:
def 一个生产装饰器的工厂(告诉我怎么生产):
    def 一个装饰器(我要被装饰):
        
        def 我就是一个增强的原函数(*args, **kwargs):
            
            print(告诉我怎么生产)
            print('给 `我要被装饰` 增加一点儿能力')
            return 我要被装饰(*args, **kwargs)
        
        # 你真的需要它么？？？？？？？？？
        def 改变生产方式(新的生产方式):
            nonlocal 告诉我怎么生产
            告诉我怎么生产 = 新的生产方式

        setattr(我就是一个增强的原函数, '改变生产方式', 改变生产方式)
        
        return 我就是一个增强的原函数
    return 一个装饰器

**1. 工厂生产装饰器**

In [107]:
一个生产装饰器的工厂('你就随便生产吧')

<function __main__.一个生产装饰器的工厂.<locals>.一个装饰器(我要被装饰)>

In [108]:
一个生产装饰器的工厂('你就随便生产吧').__name__

'一个装饰器'

In [109]:
def somefunc():
    print('I am somefunc')

**2. 把函数交给装饰器**

In [110]:
一个生产装饰器的工厂('你就随便生产吧')(somefunc)

<function __main__.一个生产装饰器的工厂.<locals>.一个装饰器.<locals>.我就是一个增强的原函数(*args, **kwargs)>

In [111]:
一个生产装饰器的工厂('你就随便生产吧')(somefunc).__name__

'我就是一个增强的原函数'

**3. 调用装饰好的函数**

In [112]:
一个生产装饰器的工厂('你就随便生产吧')(somefunc)()

你就随便生产吧
给 `我要被装饰` 增加一点儿能力
I am somefunc


In [113]:
newfunc = 一个生产装饰器的工厂('你就随便生产吧')(somefunc)

In [114]:
newfunc()

你就随便生产吧
给 `我要被装饰` 增加一点儿能力
I am somefunc


**4. 更改生产方式**

In [115]:
newfunc.改变生产方式('给我好好生产')

In [116]:
newfunc()

给我好好生产
给 `我要被装饰` 增加一点儿能力
I am somefunc


**@装饰器**(没有括号)     
**@工厂()**(有括号) 

In [117]:
@一个生产装饰器的工厂('我有一个特别特殊的需求')
def somefunc():
    print('I am somefunc')

In [118]:
somefunc()

我有一个特别特殊的需求
给 `我要被装饰` 增加一点儿能力
I am somefunc


### **But Why?**

**那么问题来了，既然这么复杂，干嘛还写装饰器啊，头顶上那个东西，根本说明不了这个函数的作用，你还要改它**              
**那你让别人怎么debug， 还要看代码，还要知道你更改过后，这个函数的调用状态**     
**为了你一时爽， 就让别人看不懂代码么？？？？？？**

<font color='darkred'>**Warning: Avoid Global [modify nonlocal] Variables**</font>                    
Programs with global variables are difficult to maintain and extend because <font color='darkred'>**you can no longer view each function as a “black box” that simply receives arguments and returns a result.**</font>        

When functions modify global variables, it becomes more <font color='drakred'>**difficult to understand the effect of function calls**</font>. As programs get larger, this difficulty mounts quickly. Instead of using global variables, use function parameter variables and return values to transfer information from one part of a program to another. 

Global constants, however, are fine. You can place them at the top of a Python source file and access (but not modify) them in any of the functions in the file. Do not use a global declaration to access constants.       

### Cases

In [12]:
import pandas as pd
import logging
import numpy as np
logging.basicConfig(level= logging.INFO)

def check_null_df(f):
    def wrapper(*args, **kwargs):
        df = f(*args, **kwargs)
        if df is None or len(df) == 0:
            logging.info(f"No rows returned by function {f.__name__}")
            position_str = ','.join(str(s) for s in args)
            keyword_str = ','.join([f'{str(k)}={str(v)}' for k,v in kwargs.items()])
            logging.info(f"{f.__name__}({position_str}, {keyword_str})")
        return df
    return wrapper

In [123]:
import logging

In [125]:
@check_null_df
def return_empty(id, name,tt):
    return pd.DataFrame()

return_empty(2, name = 'wanghuan',tt = 888)

@check_null_df
def not_empty(id, name,tt):
    return pd.DataFrame(np.random.random((3,4)))

not_empty(id = 1,name = 'wanghuan',tt = 22)

INFO:root:No rows returned by function return_empty
INFO:root:return_empty(2, name=wanghuan,tt=888)


,0,1,2,3
0,0.846974,0.113276,0.744018,0.504474
1,0.853213,0.803645,0.993572,0.290022
2,0.764608,0.827111,0.482522,0.331674


In [145]:
from functools import wraps

In [156]:
def check_missing_df(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        df = f(*args, **kwargs)
        if df.isnull().sum().sum() > 0:
            position_str = ','.join(str(s) for s in args)
            keyword_str = ','.join([f'{str(k)}={str(v)}' for k,v in kwargs.items()])
            logging.info('Dataframe returned contain missing values')
            logging.info(f"With missing call -- {f.__name__}({position_str}, {keyword_str})")
        return df
    return wrapper

In [157]:
@check_null_df
@check_missing_df
def df_with_missing(uid, server):
    '''I have some documentation here'''
    return pd.DataFrame([[np.nan,1],[2,3]])

In [158]:
df_with_missing(3, server = 123)

INFO:root:Dataframe returned contain missing values
INFO:root:With missing call -- df_with_missing(3, server=123)


,0,1
0,NaN,1
1,2.0,3


In [165]:
def add_day_id(df, group_col = 'from_server', date_col = 'date', id_offset = 0, day_id_name = 'day_id'):
    result = []
    unique_servers = sorted(df[group_col].unique())

    for server_id in unique_servers:
        unique_dates = sorted(df[df.from_server == server_id ][date_col].unique())
        result += [[date, i + id_offset + 1, server_id ] for i, date in enumerate(unique_dates)]

    result = pd.DataFrame(result, columns = [date_col,day_id_name,group_col])
    return df.merge(result, on = [date_col, group_col], how = 'inner')


def with_day_id(*oargs, **okwargs):
    '''
        Allow parameters:
        group_col: like server_id, from_server
        date_col: YYYY-MM-DD like
        id_offset: day_id startswith
        day_id_name: rename day_id column
    '''
    def decorator(f):
        @wraps(f)
        def wrapper(*args, **kwargs):
            df = f(*args, **kwargs)
            return add_day_id(df,*oargs,**okwargs)
        return wrapper
    return decorator

@with_day_id(id_offset = 4)
def get_df():
    from datetime import datetime
    from_server = np.random.choice([1,2,3], size = 100)
    dates = [datetime(2020,5,np.random.choice(np.arange(1,6))).date() for i in range(100)]
    df = pd.DataFrame(zip(from_server, dates), columns = ['from_server','date'])
    return df


print(get_df().day_id.unique())

[9 5 6 7 8]


In [15]:
from functools import wraps
import functools
import inspect
def check_is_admin(f):
    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        func_args = inspect.getcallargs(f, *args, **kwargs)
        print(func_args)
        if func_args.get('username') != 'admin':
            raise Exception("This user is not allowed to get food")
        return f(*args, **kwargs)
    return wrapper

In [16]:
@check_is_admin
def somefunc(a,b,c, username = 'admin'):
    print('haha')

In [17]:
somefunc(1,2,3)

{'a': 1, 'b': 2, 'c': 3, 'username': 'admin'}
haha


In [23]:
def check_missing_df(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        func_args = inspect.getcallargs(f, *args, **kwargs)
        df = f(*args, **kwargs)
        if df.isnull().sum().sum() > 0:
            logging.info('Dataframe returned contain missing values')
            parameters = ",".join([f'{k} = {v}'for k,v in func_args.items()])
            logging.info(f"{f.__name__}({parameters})")
        return df
    return wrapper

@check_missing_df
def df_with_missing(uid, server):
    '''I have some documentation here'''
    return pd.DataFrame([[np.nan,1],[2,3]])

In [24]:
df_with_missing(123,123)

INFO:root:Dataframe returned contain missing values
INFO:root:df_with_missing(uid = 123,server = 123)


,0,1
0,NaN,1
1,2.0,3


# 工具类

1. 所有的项目依赖Dbtools， Dbtools的功能基本没啥大变化了，稳定下来了，单独写成了一个工具类             
2. 抽取出来一些功能性的东西，比如utils， 这些和数据源基本没有什么大关系的东西，单独写成一个工具类 
3. queries可能以后要被复用的，单独写成一个queries，就是原来query_XXXX的那些东西
4. 大家都一直复用的东西，比如user_type写成了一个工具类

```bash
├─dbx
│  │  .gitignore
│  │  README.md
│  │  requirements.txt
│  │  setup.py
│  │
│  └─dbx
│          config.py
│          dbtools.py
│          db_log_tools.py
│          increment.py
│          parallel.py
│          __init__.py
```


# 项目
1. 运行某个特定的业务场景 - KPI， t表， 模型什么的
2. 某一个看着像功能，但是里面又100000000个sql的东西，基本像写死了的东西
**随便写，最后有一个程序入口就行了， 不管你是一个文件夹一堆东西，还是什么什么的，最后有一个能跑起来的东西就行了**    
你可以是一个文件夹，你就不一定要打包，你也可以import自己的文件夹里面的其他东西的

`app.py`
```python

from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/store',methods = ['POST'])
def create_store():
    pass

@app.route('/store/<string:name>',methods = ['GET'])
def get_store(name):
    pass

@app.route('/store/<string:name>/item',methods = ['GET'])
def get_items_in_store(name):
    pass

app.run(port = 5000)
```

或者，就是一段简单的python程序

```python
if __name__ =='__main__':
    print('Hello world')

```


#### 你只有一个.py的情况下，不用新建一个项目，包含再一个叫什么什么报表的Repo就好了


# 打包

`https://github.com/Camel-Data/document/blob/main/pypi/pypi%20documentation.md`

```bash
pip install pack_name
pip install pack_name --upgrade
```

# 版本

同一个版本，不能多次上传到Pypi私服上面去的，你要更新一下版本号


### 先手动删除一下build， dist， egg_info我搞明白自动删除了告诉你们

```python
from setuptools import setup


setup(name='dbx',
      version='0.0.4',
      description='Database connection tools',
      author='My name is Barry Ellen, The fastest man alive',
      packages=['dbx'],
      install_requires=['pymysql','pandas','tqdm',
      'sqlalchemy','pyathena','matplotlib'])

```



```bash
C:.
│   .gitignore
│   README.md
│   requirements.txt
│   setup.py
│
├───build
│   ├───bdist.win-amd64
│   └───lib
│       └───dbx
│               config.py
│               dbtools.py
│               db_log_tools.py
│               increment.py
│               parallel.py
│               __init__.py
│
├───dbx
│   │   config.py
│   │   dbtools.py
│   │   db_log_tools.py
│   │   increment.py
│   │   parallel.py
│   │   __init__.py
│
├───dbx.egg-info
│       dependency_links.txt
│       PKG-INFO
│       requires.txt
│       SOURCES.txt
│       top_level.txt
│
└───dist
        dbx-0.0.4-py3-none-any.whl
        dbx-0.0.4.tar.gz

```

```bash
T_table_monitor_aoz.py
T_table_abnormals_monitor_wao.py
T_table_abnormals_monitor_wof.py  
KPI_abnormals_monitor_aoz.py
KPI_abnormals_monitor_wao.py
KPI_abnormals_monitor_wof.py

```